In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping)
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

2024-06-26 18:28:00.819417: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 18:28:00.846494: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 18:28:00.847099: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 18:28:01.554862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
# 修改anchor生成函数
def generate_ssd_anchors(num_layers, min_scale, max_scale, aspect_ratios, reduce_boxes_in_lowest_layer):
    scales = np.linspace(min_scale, max_scale, num_layers)
    anchors = []

    for i, scale in enumerate(scales):
        if i == 0 and reduce_boxes_in_lowest_layer:
            # 在最浅层使用减少后的 boxes
            box = [0.5, 0.5, scale * np.sqrt(aspect_ratios[0]), scale / np.sqrt(aspect_ratios[0])]
            anchors.append(box)
        else:
            for aspect_ratio in aspect_ratios:
                width = scale * np.sqrt(aspect_ratio)
                height = scale / np.sqrt(aspect_ratio)
                box = [0.5, 0.5, width, height]
                anchors.append(box)

    return np.array(anchors)


# 生成anchors
num_layers = 5
min_scale = 0.10
max_scale = 0.85
aspect_ratios = [1.0, 2.0, 0.5, 3.0, 0.3333]
reduce_boxes_in_lowest_layer = True

anchors = generate_ssd_anchors(num_layers, min_scale, max_scale, aspect_ratios, reduce_boxes_in_lowest_layer)
print("Generated anchors:", anchors)

Generated anchors: [[0.5        0.5        0.1        0.1       ]
 [0.5        0.5        0.2875     0.2875    ]
 [0.5        0.5        0.4065864  0.2032932 ]
 [0.5        0.5        0.2032932  0.4065864 ]
 [0.5        0.5        0.49796461 0.1659882 ]
 [0.5        0.5        0.1659799  0.49798951]
 [0.5        0.5        0.475      0.475     ]
 [0.5        0.5        0.67175144 0.33587572]
 [0.5        0.5        0.33587572 0.67175144]
 [0.5        0.5        0.82272413 0.27424138]
 [0.5        0.5        0.27422767 0.82276527]
 [0.5        0.5        0.6625     0.6625    ]
 [0.5        0.5        0.93691649 0.46845824]
 [0.5        0.5        0.46845824 0.93691649]
 [0.5        0.5        1.14748366 0.38249455]
 [0.5        0.5        0.38247543 1.14754104]
 [0.5        0.5        0.85       0.85      ]
 [0.5        0.5        1.20208153 0.60104076]
 [0.5        0.5        0.60104076 1.20208153]
 [0.5        0.5        1.47224319 0.49074773]
 [0.5        0.5        0.49072319 1.4723

In [5]:
def get_classes(classes_path):
    with open(classes_path, encoding='utf-8') as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names, len(class_names)

def get_img_output_length(height, width):
    feature_heights = [15, 8, 4, 2, 1]
    feature_widths = [20, 10, 5, 3, 1]
    return np.array(feature_heights), np.array(feature_widths)

class AnchorBox():
    def __init__(self, input_shape=[120, 160], min_size=None, max_size=None, aspect_ratios=None):
        self.input_shape = input_shape
        self.min_size = min_size
        self.max_size = max_size
        self.aspect_ratios = []
        for i in aspect_ratios:
            self.aspect_ratios.append(i)
            if i != 1:
                self.aspect_ratios.append(1.0 / i)

    def __call__(self, layer_shape):
        layer_height = layer_shape[0]
        layer_width = layer_shape[1]
        img_height = self.input_shape[0]
        img_width = self.input_shape[1]
        
        box_widths = []
        box_heights = []
        for i in self.aspect_ratios:
            if i == 1 and len(box_widths) == 0:
                box_widths.append(self.min_size)
                box_heights.append(self.min_size)
            elif i == 1 and len(box_widths) > 0:
                box_widths.append(np.sqrt(self.min_size * self.max_size))
                box_heights.append(np.sqrt(self.min_size * self.max_size))
            elif i != 1:
                box_widths.append(self.min_size * np.sqrt(i))
                box_heights.append(self.min_size / np.sqrt(i))
        
        box_widths = 0.5 * np.array(box_widths)
        box_heights = 0.5 * np.array(box_heights)
        
        step_x = img_width / layer_width
        step_y = img_height / layer_height
        linx = np.linspace(0.5 * step_x, img_width - 0.5 * step_x, layer_width)
        liny = np.linspace(0.5 * step_y, img_height - 0.5 * step_y, layer_height)
        
        centers_x, centers_y = np.meshgrid(linx, liny)
        centers_x = centers_x.reshape(-1, 1)
        centers_y = centers_y.reshape(-1, 1)
        
        num_anchors_ = len(self.aspect_ratios)
        anchor_boxes = np.concatenate((centers_x, centers_y), axis=1)
        anchor_boxes = np.tile(anchor_boxes, (1, 2 * num_anchors_))
        
        anchor_boxes[:, ::4] -= box_widths
        anchor_boxes[:, 1::4] -= box_heights
        anchor_boxes[:, 2::4] += box_widths
        anchor_boxes[:, 3::4] += box_heights
        
        anchor_boxes[:, ::2] /= img_width
        anchor_boxes[:, 1::2] /= img_height
        anchor_boxes = anchor_boxes.reshape(-1, 4)

        anchor_boxes = np.minimum(np.maximum(anchor_boxes, 0.0), 1.0)
        return anchor_boxes

def get_anchors(input_shape=[120, 160], anchors_size=[32, 59, 86, 113, 140]):
    feature_heights, feature_widths = get_img_output_length(input_shape[0], input_shape[1])
    aspect_ratios = [[1], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]
    anchors = []
    for i in range(len(feature_heights)):
        anchors.append(AnchorBox(input_shape, anchors_size[i], max_size=anchors_size[i+1] if i+1 < len(anchors_size) else None, aspect_ratios=aspect_ratios[i])
                       ([feature_heights[i], feature_widths[i]]))
    anchors = np.concatenate(anchors, axis=0)
    return anchors

result = get_anchors()
print(result)

[[0.         0.         0.125      0.16666667]
 [0.         0.         0.175      0.16666667]
 [0.025      0.         0.225      0.16666667]
 ...
 [0.19064078 0.         0.80935922 1.        ]
 [0.         0.16321234 1.         0.83678766]
 [0.24740926 0.         0.75259074 1.        ]]
